# Doesnt work yet

In [9]:
import pyvista as pv
import numpy as np
import xarray as xr
from JulesD3D.utils import quickDF
from JulesD3D.cleanNetCDF import fixMeshGrid, makeBottomStress,addDepth, makeVelocity, addUnderlayerCoords
from JulesD3D.plotPyVista import makeBottomSurface, makeStructuredGridDepth, easyTimeFormat
from cmocean.cm import deep
import matplotlib.pyplot as plt
from JulesD3D.utils import ncFilepicker

In [10]:
folder = '/Volumes/Seagate Expansion Drive/ThesisResults/Current'

nc_filename = ncFilepicker(folder)

In [11]:
nc_filename

Select(description='NetCDF file:', layout=Layout(height='120px', width='90%'), options=('/Volumes/Seagate Expa…

In [13]:
basename = path.basename(nc_filename.value)
plot_title, _ = path.splitext(basename)
plot_title

'Slope1.25_30min_conc3.0_2575_discharge4400_SMALLER_timestep_MORFAC_3hrs_150mChannelNEW'

In [14]:
trim = xr.open_dataset(nc_filename.value) # Open with xarray : DataSet : A collection of DataArrays

In [15]:
if 'velocity' not in trim:
    trim = fixMeshGrid(trim, trim.XZ.values, trim.YZ.values, mystery_flag=True)
#     trim = makeBottomStress(trim)    
    trim = addDepth(trim)
#     trim = makeVelocity(trim)
    trim = addUnderlayerCoords(trim)
else:
    print("DataSet is preprocessed, 'velocity' is already in DataSet")

62 x 152 grid
XZ (152, 62)
YZ (152, 62)


### Process data vars

In [6]:
concentration = trim.R1.transpose('time', 'LSTSCI', 'M', 'N', 'KMAXOUT_RESTR')
# concentration_silt = concentration.isel(LSTSCI=1)
concentration_silt = concentration.isel(LSTSCI=1)
concentration_silt_at_time = concentration_silt.isel(time=5)
concentration_silt_at_time = concentration_silt_at_time.assign_attrs({'long_name': 'Silt concentrations percentage per layer in zeta point',
                                                              'units': '%',
                                                              'grid': 'grid',
                                                              'location': 'face'})

In [7]:
# Reorder velocity dims to match x, y, z
velocity = trim.velocity.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
velocity.dims

AttributeError: 'Dataset' object has no attribute 'velocity'

In [21]:
density = trim.RHO.transpose('time', 'M', 'N', 'KMAXOUT_RESTR')
density.dims

('time', 'M', 'N', 'KMAXOUT_RESTR')

### Turn Delft3D output into PyVista grid

In [22]:
bottom_surface = makeBottomSurface(trim)
bottom_surface

62 x 152 grid
XZ (152, 62)
YZ (152, 62)


StructuredGrid,Information
N Cells,9000
N Points,9211
X Bounds,"0.000e+00, 1.800e+04"
Y Bounds,"1.000e+02, 4.510e+04"
Z Bounds,"-7.310e+02, -1.500e+02"
Dimensions,"61, 151, 1"
N Arrays,0


In [23]:
depth_centers_grid = makeStructuredGridDepth(trim, keyword='SIG_LYR')

'depth' DataArray found found in DataSet!
xyz_layers.shape (753920, 3)


In [24]:
initial_time = 5
time_lims = (initial_time, trim.time.size-1)
time_lims

(5, 170)

In [25]:
depth_centers_grid['scalars'] = density.isel(time=initial_time).values.ravel()

In [26]:
def callback(val):
    outputstep = int(val)
    
    scalars = density.isel(time=outputstep).values.ravel()
    print(scalars.max())
#     concentration_silt.isel(time=outputstep).values.ravel()
    p.update_scalars(scalars, mesh=depth_centers_grid)

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(bottom_surface, cmap=deep)
p.add_mesh(depth_centers_grid, opacity=0.9)
p.add_slider_widget(callback, time_lims, value=initial_time)
p.enable_depth_peeling()
p.show_grid()
p.set_scale(zscale=25)
panel = p.show()

1073.75
1116.0184
1143.3165
1042.5939
1116.0184
1120.8667
1041.9491
1031.5994
1159.8336
1066.3567
1088.1426
1111.038
1143.3165
1099.1155
1047.0826
1042.3501
1042.3501
1069.9487
1069.9487
1133.7102
1061.011
1136.3813


In [ ]:
alg = vtk.vtkThreshold()
alg.SetInputDataObject(depth_centers_grid)
# alg.SetInputArrayToProcess(0, 0, 0, field, scalars) # args: (idx, port, connection, field, name)
# alg.SetUseContinuousCellRange(continuous)

In [ ]:
def testCallback(val):
    outputstep = int(val)
    
    scalars = density.isel(time=outputstep).values.ravel()
    print(scalars.max())
#     concentration_silt.isel(time=outputstep).values.ravel()
    p.update_scalars(scalars, mesh=depth_centers_grid)
#     print("Updated scalars", scalars.size)
#     alg.SetInputDataObject(depth_centers_grid)
#     threshold_mesh = pv.wrap(alg.GetOutput())
    
#     alg.ThresholdByLower(value)
#     alg.update()

In [ ]:
# def oldCallback(val):
#     outputstep = int(val)
#     scalars = concentration_silt.isel(time=outputstep).values.ravel()
#     depth_centers_grid['conc_silt'] = scalars
    
#     thresh_conc_silt = depth_centers_grid.threshold(conc_silt_lims, scalars="conc_silt")

#     threshed_actor = p.add_mesh(thresh_conc_silt, scalars='conc_silt', cmap="plasma",
#                        opacity=0.8, clim=conc_silt_lims, ambient=0.2)
#     print("outputstep", outputstep)
    

In [ ]:
p = pv.Plotter(notebook=False)
p.add_mesh(bottom_surface, cmap=deep)
p.add_mesh(depth_centers_grid, opacity=0.9)
# p.add_mesh(thresh_conc_silt, scalars='conc_silt', cmap="plasma", opacity=0.8, clim=conc_silt_lims, ambient=0.2)
# p.add_mesh_threshold(thresh_conc_silt, scalars='conc_silt', cmap="plasma", opacity=0.8, clim=conc_silt_lims, ambient=0.2)
# p.add_slider_widget(testCallback, time_lims, value=5)
p.add_slider_widget(testCallback, time_lims, value=5)
p.enable_depth_peeling()
p.show_grid()
p.set_scale(zscale=25)
panel = p.show()